<a href="https://colab.research.google.com/github/osullik/ic23050/blob/main/DataDictionary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [142]:
from google.colab import drive
import pandas as pd
import json
from collections import defaultdict

In [143]:
DD = pd.ExcelFile("https://github.com/osullik/ic23050/blob/main/L4%20Washington%20Fatal%20Crash%20Files%20-%20Washington%20Traffic%20Safety%20Commission/DataDictionary_Washington%20Fatal%20Crash%20Files.xlsx?raw=true")

In [144]:
for sheet in DD.sheet_names:
  print(sheet)
  


ReadMe
CRASH Data Files
PERSON Data Files
Tables1
Tables2
Tables3
Tables4
Tables5
Tables6
Tables7
Tables8
Tables9


In [249]:
#Seperate out each sheet in the data dictionary 
ReadMe = pd.read_excel(DD, 'ReadMe')
CrashDataFiles = pd.read_excel(DD, 'CRASH Data Files')
PersonDataFiles = pd.read_excel(DD, 'PERSON Data Files')
#Tables1 = pd.read_excel(DD, 'Tables1')
Tables2 = pd.read_excel(DD, 'Tables2')
Tables3 = pd.read_excel(DD, 'Tables3')
Tables4 = pd.read_excel(DD, 'Tables4')
Tables5 = pd.read_excel(DD, 'Tables5')
Tables6 = pd.read_excel(DD, 'Tables6')
Tables7 = pd.read_excel(DD, 'Tables7')
Tables8 = pd.read_excel(DD, 'Tables8')
Tables9 = pd.read_excel(DD, 'Tables9')

In [321]:
'''
getKVPairs is a function to extract KV Pairs from dirty excel data
  Input Args:
    desc: a String "description" of the values we want to extract. Should 
      include target KV pairs in the form KEY=VALUE where KEY is a number and 
      VALUE are non-numeric substrings. 
  Method: 
    Break the string into characters
    Accumulate numeric values until an "=" is seen. The "=" is an assignment  
      we infer the numbers prior are the key and this is appended to keyList
    Accumulate characters until either EOF or another numerical value is found
      we infer all strings between keys are their associated values. 
    
    Before appending a value to the definition list, remove newline and 
      truncate spaces

    Zip the KeyList and DefinitionList together, and return

  Returns:
    A list of Tuples in the form [(Key, Value)]

'''
def getKVPairs(desc):
                                                #Init Vars
  number = ""
  definition = ""
  keyList = []
  definitionList = []
  keyFound = False
                                                #Filter out non-strings
  if type(desc) == str:
    for char in desc:
      
      if char.isnumeric() == True:              #Build the Keys
        if keyFound == True:
          definition = definition.strip("\n")
          definition = definition.strip()
          definitionList.append(definition)
          definition = ""
          keyFound = False
          number += char

        else:                                   #Build the Values
          number += char

      elif char == ("="):                       #Commit the Key
        keyList.append(number)
        number = ""
        keyFound = True

      else:                                     #Build the value
        definition += char 
    else:
      pass
                                                #Add the last Definition
    definition = definition.strip("\n")
    definition = definition.strip()
    definitionList.append(definition)

  toReturn = list(zip(keyList,definitionList))  #Create a list of (K,V) Tuples

  return(toReturn)    

In [335]:
#Parse the CrashDataFile into JSON
dataDict = defaultdict(dict)                    #Build the Data Dictionary

for idx, series in CrashDataFiles.iterrows():   #Step through each var in turn
  var = series['VARIABLE']
  typ = series['TYPE']
  length = series['LENGTH']
  format = series['FORMAT']
  label = series['LABEL']
  desc = series['DESCRIPTION/COMMENT']

                                                #Populate the Data Dictionary
  dataDict[var]["TYPE"]     = typ
  dataDict[var]["LENGTH"]   = length 
  dataDict[var]["FORMAT"]   = format 
  dataDict[var]["LABEL"]    = label 

  kvPairList = getKVPairs(desc)                 #Get list (K,V) tuples for DATA

  dataDict[var]["DATA"]    = dict(kvPairList)

  


In [428]:
#Create JSON object out of CrashFileData
json_CrashDataFile = json.dumps(dataDict, indent = 4) 

In [375]:
#Parse the PersonDataFile into JSON
dataDict_PersonData = defaultdict(dict)                    #Build the Data Dictionary

print(PersonDataFiles.columns)

for idx, series in PersonDataFiles.iterrows():   #Step through each var in turn
  var = series['VARIABLE']
  typ = series['TYPE']
  length = series['LENGTH']
  format = series['FORMAT']
  label = series['LABEL']
  desc = series['DESCRIPTION/COMMENT']

                                                #Populate the Data Dictionary
  dataDict_PersonData[var]["TYPE"]     = typ
  dataDict_PersonData[var]["LENGTH"]   = length 
  dataDict_PersonData[var]["FORMAT"]   = format 
  dataDict_PersonData[var]["LABEL"]    = label 

  kvPairList = getKVPairs(desc)                 #Get list (K,V) tuples for DATA

  dataDict_PersonData[var]["DATA"]    = dict(kvPairList)

Index(['Unnamed: 0', 'VARIABLE', 'TYPE', 'LENGTH', 'FORMAT', 'LABEL',
       'DESCRIPTION/COMMENT', 'Notes / SAS Code', 'Unnamed: 8', 'Unnamed: 9'],
      dtype='object')


In [429]:
#Create JSON object out of PersonFileData
json_PersonDataFile = json.dumps(dataDict_PersonData, indent = 4) 

In [431]:
#Extract the messy table data from the TABLES1 Sheet 
  #(Manual inspection used to determine the cut points for the DFs)
Tables1_county = pd.read_excel(DD, 'Tables1', skiprows = 1, nrows=40,  usecols= 'A')
Tables1_harmfulEvent = pd.read_excel(DD, 'Tables1', skiprows = 1, nrows=40,  usecols= 'B,D')
Tables1_trafficControl = pd.read_excel(DD, 'Tables1', skiprows = 42, nrows=20,  usecols= 'A,D')
Tables1_seatingPosition = pd.read_excel(DD, 'Tables1', skiprows = 0, nrows=31,  usecols= 'E')


In [432]:
#Parse out the County Numbering System
dataDict_Table1 = defaultdict(dict)                    #Build the Data Dictionary

#print(Tables1_county.columns)
dataDict_Table1["County_Name"]["DATA"] = {}

for idx, series in Tables1_county.iterrows():   #Step through each var in turn
  
  desc = series['county_char']

  kvPairList = getKVPairs(desc)                 #Get list (K,V) tuples for DATA
  kvPairTuple = tuple(kvPairList[0])

  dataDict_Table1["County_Name"]["DATA"][kvPairTuple[0]] = kvPairTuple[1]

In [423]:
#Parse out the Harmful Event Data from Table1

dataDict_Table1["Harmful_Event"]["DATA"] = {}

for idx, series in Tables1_harmfulEvent.iterrows():   #Step through each var in turn
  
  desc1 = series['Non-Collision']
  desc2 = series['Unnamed: 3']

  kvPairList1 = getKVPairs(desc1)                 #Get list (K,V) tuples for DATA
  kvPairList2 = getKVPairs(desc2)


  #Following try/catch statements account for misalignment of columns in 
  #input data
  try:                                            #Transform into tuples
    kvPairTuple1 = tuple(kvPairList1[0])
  except IndexError:
    pass

  try:
    kvPairTuple2 = tuple(kvPairList2[0])
  except IndexError:
    pass

  try:                                           #Update the Dicts
    dataDict_Table1["Harmful_Event"]["DATA"][kvPairTuple1[0]] = kvPairTuple1[1]
  except NameError:
    pass
  try:
    dataDict_Table1["Harmful_Event"]["DATA"][kvPairTuple2[0]] = kvPairTuple2[1]
  except NameError:
    pass

In [425]:
#Parse out the Traffic Control Data from Table1

dataDict_Table1["Traffic_Control"]["DATA"] = {}

for idx, series in Tables1_trafficControl.iterrows():   #Step through each var in turn
  
  desc1 = series['TRAFFIC CONTROL DEVICE']
  desc2 = series['Unnamed: 3']

  kvPairList1 = getKVPairs(desc1)                 #Get list (K,V) tuples for DATA
  kvPairList2 = getKVPairs(desc2)

  #Following try/catch statements account for misalignment of columns in 
  #input data
  try:                                            #Transform into tuples
    kvPairTuple1 = tuple(kvPairList1[0])
  except IndexError:
    pass

  try:
    kvPairTuple2 = tuple(kvPairList2[0])
  except IndexError:
    pass

  try:                                           #Update the Dicts
    dataDict_Table1["Traffic_Control"]["DATA"][kvPairTuple1[0]] = kvPairTuple1[1]
  except NameError:
    pass
  try:
    dataDict_Table1["Traffic_Control"]["DATA"][kvPairTuple2[0]] = kvPairTuple2[1]
  except NameError:
    pass

In [426]:
#Parse out the Seating Positions
dataDict_Table1["Seating_Position"]["DATA"] = {}

for idx, series in Tables1_seatingPosition.iterrows():   #Step through each var in turn
  
  desc = series['SEATING POSITION']

  kvPairList = getKVPairs(desc)                 #Get list (K,V) tuples for DATA
  try:
    kvPairTuple = tuple(kvPairList[0])
    dataDict_Table1["Seating_Position"]["DATA"][kvPairTuple[0]] = kvPairTuple[1]
  except IndexError:
    pass

In [433]:
#Create a JSON object of the Table1 Data
json_Table1 = json.dumps(dataDict_Table1, indent = 4) 